# Extract Seurat matrices

We were unable to convert the Seurat objects to loom or to anndata, so let's just extract the raw and normalised matrices and try to do a naive analysis with them

In [2]:
library(Seurat)

In [3]:
ecs = readRDS("~/Downloads/bismark/Electroporated_cells_seur.RData")
mcs = readRDS("~/Downloads/bismark/microinjected_cells_seur.RData")

In [4]:
ecs

An object of class Seurat 
788 features across 711 samples within 1 assay 
Active assay: RNA (788 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap

In [5]:
mcs

An object of class Seurat 
789 features across 589 samples within 1 assay 
Active assay: RNA (789 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap

In [11]:
ecs.clusters <- ecs$seurat_clusters
mcs.clusters <- mcs$seurat_clusters

In [22]:
str(ecs, max.level = 2)

Formal class 'Seurat' [package "Seurat"] with 13 slots


Warning message:
“Not a validObject(): no slot of name "images" for this object of class "Seurat"”


  ..@ assays      :List of 1
  ..@ meta.data   :'data.frame':	711 obs. of  50 variables:
  ..@ active.assay: chr "RNA"
  ..@ active.ident: Factor w/ 6 levels "Apical progenitors",..: 1 1 4 1 1 3 3 6 1 2 ...
  .. ..- attr(*, "names")= chr [1:711] "EUEE14DMSORP5_1" "EUEE14DMSORP5_2" "EUEE14DMSORP5_3" "EUEE14DMSORP5_4" ...
  ..@ graphs      :List of 2
  ..@ neighbors   : list()
  ..@ reductions  :List of 2
  ..@ project.name: chr "SingleCellExperiment"
  ..@ misc        : list()
  ..@ version     :Classes 'package_version', 'numeric_version'  hidden list of 1
  ..@ commands    :List of 7
  ..@ tools       : list()
  ..@ NA          : NULL


In [4]:
ecs.labels <- ecs@active.ident
mcs.labels <- mcs@active.ident

In [5]:
# This includes: condition, cluster identity, lib size, various pct amounts
ecs.data <- ecs@meta.data
mcs.data <- mcs@meta.data

In [6]:
ecs.matrix <- ecs@assays$RNA
mcs.matrix <- mcs@assays$RNA

In [36]:
str(ecs.matrix, max.level = 2)

Formal class 'Assay' [package "Seurat"] with 8 slots


Warning message:
“Not a validObject(): no slot of name "assay.orig" for this object of class "Assay"”


  ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ data         : num [1:788, 1:711] 0 0 5.11 1.46 0 ...
  .. ..- attr(*, "dimnames")=List of 2
  ..@ scale.data   : num [1:788, 1:711] -1.093 -0.569 2.363 -0.799 -1.141 ...
  .. ..- attr(*, "dimnames")=List of 2
  ..@ key          : chr "rna_"
  ..@ var.features : chr(0) 
  ..@ meta.features:'data.frame':	788 obs. of  0 variables
  ..@ misc         : symbol NULL
  ..@ NA           : NULL


In [7]:
ecs.counts <- ecs.matrix@data
mcs.counts <- mcs.matrix@data
ecs.scaled <- ecs.matrix@scale.data
mcs.scaled <- mcs.matrix@scale.data

It's unclear if the counts have been normalised or have had Grün's count-correction applied...  
#sort(colSums(ecs.counts),decreasing = T)

## Export all matrices

In [29]:
dir.create("files//4_extract//")

In [9]:
writer <- function(data, fname){
    write.table(t(data), file = fname, row.names = T, sep = "\t", dec = ".")
}

writer(ecs.data, "files/4_extract/ecs.metadata.tsv")
writer(mcs.data, "files/4_extract/mcs.metadata.tsv") 
writer(ecs.counts, "files/4_extract/ecs.counts.tsv") 
writer(mcs.counts, "files/4_extract/mcs.counts.tsv") 
writer(ecs.scaled, "files/4_extract/ecs.scaled.tsv") 
writer(mcs.scaled, "files/4_extract/mcs.scaled.tsv") 

We will process the rest of these in ScanPy, either doing a naive analysis on the RAW matrices, or by utilising the meta to bypass the pre-processing